# Evaluate Dst forecasts

> The baseline is the benchmark by Licata et. al.

In [6]:
# Imports
import sys
sys.path.append('..')
from tsai.basics import *
from sklearn.metrics import mean_squared_error, mean_absolute_error
from swdf.utils import *
import wandb
wandb_api = wandb.Api()

In [7]:
# Constants
ARTIFACT_DOWNLOAD_PATH = Path(os.environ["WANDB_DIR"])/"wandb/artifacts/solfsmy_eval_tmp"

In [8]:
# Config
config = yaml2dict('config/dst.yaml')
config = config.eval
config

```json
{ 'dst_data_path': '../data/DST_IAGA2002.txt',
  'learner_artifact': None,
  'solfsmy_data_path': '../data/SOLFSMY.TXT'}
```

In [9]:
if config.learner_artifact is None:
    learner_path = 'tmp'
else:
    learner_path = wandb_api.artifact(config.learner_artifact).download(root=ARTIFACT_DOWNLOAD_PATH)
learn = load_learner_all('tmp', verbose=True, device='cpu')

Learner loaded:
path          = 'tmp'
dls_fname     = '['dls_0.pth', 'dls_1.pth', 'dls_2.pth', 'dls_3.pth']'
model_fname   = 'model.pth'
learner_fname = 'learner.pkl'


In [10]:
y_test_preds, y_test = learn.get_preds(ds_idx = 2, with_targs=True)
y_test_preds = to_np(y_test_preds)
y_test = to_np(y_test)
print(f"y_test_preds.shape: {y_test_preds.shape}")     

y_test_preds.shape: (45888, 1, 144)


In [11]:
horizon = y_test.shape[-1]
data_columns_fcst = ['DST']

The threshold classify the forecasts of Dst are based on the values of the Dst itself
and the solar activity level, as defined by the F10.7 (according to Licata et al.).
Therefore, below is the table of all possible combined conditions to evaluate Dst.

| Dst Level | Solar Level | Dst Value         | Solar Value       |
|-----------|-------------|-------------------|-------------------|
| G0        | Low         | Dst ≥ -30         | F 10.7 ≤ 75       |
| G0        | Moderate    | Dst ≥ -30         | 75 < F 10.7 ≤ 150 |
| G0        | Elevated    | Dst ≥ -30         | 150 < F 10.7 ≤ 190|
| G0        | High        | Dst ≥ -30         | F 10.7 > 190      |
| G1        | Low         | -30 > Dst ≥ -50   | F 10.7 ≤ 75       |
| G1        | Moderate    | -30 > Dst ≥ -50   | 75 < F 10.7 ≤ 150 |
| G1        | Elevated    | -30 > Dst ≥ -50   | 150 < F 10.7 ≤ 190|
| G1        | High        | -30 > Dst ≥ -50   | F 10.7 > 190      |
| G2        | Low         | -50 > Dst ≥ -90   | F 10.7 ≤ 75       |
| G2        | Moderate    | -50 > Dst ≥ -90   | 75 < F 10.7 ≤ 150 |
| G2        | Elevated    | -50 > Dst ≥ -90   | 150 < F 10.7 ≤ 190|
| G2        | High        | -50 > Dst ≥ -90   | F 10.7 > 190      |
| G3        | Low         | -90 > Dst ≥ -130  | F 10.7 ≤ 75       |
| G3        | Moderate    | -90 > Dst ≥ -130  | 75 < F 10.7 ≤ 150 |
| G3        | Elevated    | -90 > Dst ≥ -130  | 150 < F 10.7 ≤ 190|
| G3        | High        | -90 > Dst ≥ -130  | F 10.7 > 190      |
| G4        | Low         | -130 > Dst ≥ -350 | F 10.7 ≤ 75       |
| G4        | Moderate    | -130 > Dst ≥ -350 | 75 < F 10.7 ≤ 150 |
| G4        | Elevated    | -130 > Dst ≥ -350 | 150 < F 10.7 ≤ 190|
| G4        | High        | -130 > Dst ≥ -350 | F 10.7 > 190      |
| G5        | Low         | Dst ≤ -350        | F 10.7 ≤ 75       |
| G5        | Moderate    | Dst ≤ -350        | 75 < F 10.7 ≤ 150 |
| G5        | Elevated    | Dst ≤ -350        | 150 < F 10.7 ≤ 190|
| G5        | High        | Dst ≤ -350        | F 10.7 > 190      |


In order to create all those combined conditions, let's merge the data from Dst
and F10.7. Since this data is provided with different frequencies (1 day for F10.7 
and 1 hour for Dst, we will fill the missing timestamps of F10.7 with forward-filling)


In [12]:
# We'll load again the data from Dst with datetimes, and 
# merge the DATE and TIME columns into a single datetime column
df_dst_raw = pd.read_csv(config.dst_data_path, sep='\s{2,}|\s', 
                         names=['DATE', 'TIME', 'DOY', 'DST'])
df_dst_raw['datetime'] = pd.to_datetime(df_dst_raw.DATE + ' ' + df_dst_raw.TIME)

# We need to load the F10.7 data to classify the DST predictions
df_solfsmy = pd.read_csv(config.solfsmy_data_path, delim_whitespace=True, comment='#', header=None, 
                 names=['Year', 'DDD', 'JulianDay', 'F10', 'F81c', 'S10', 'S81c', 
                        'M10', 'M81c', 'Y10', 'Y81c', 'Ssrc'])

# # Convert the JulianDay column to a datetime column
df_solfsmy['datetime'] = pd.to_datetime(df_solfsmy['JulianDay'], unit='D', origin='julian')
# Shift the datetime from 12:00 to 00:00
df_solfsmy['datetime'] = df_solfsmy['datetime'] - pd.Timedelta(hours=12)

# Merge the DST and F10.7 dataframes on the datetime column, and keep only the columns we need
df_combined = pd.merge(df_dst_raw, df_solfsmy, on='datetime', how='left')
df_combined = df_combined[['datetime', 'DST', 'F10']]

# The F10 is given only at 12:00 (00:00 now that we shifted), so we'll forward 
# fill the rest of the values
df_combined['F10'] = df_combined['F10'].fillna(method='ffill')
df_combined.head()

/tmp/ipykernel_65388/1854806815.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_dst_raw = pd.read_csv(config.dst_data_path, sep='\s{2,}|\s',


,datetime,DST,F10
0,1997-01-01 00:00:00,-5.0,72.4
1,1997-01-01 01:00:00,0.0,72.4
2,1997-01-01 02:00:00,3.0,72.4
3,1997-01-01 03:00:00,6.0,72.4
4,1997-01-01 04:00:00,9.0,72.4


In [75]:
X_combined, y_combined = prepare_forecasting_data(df_combined, fcst_history=learn.dls.len, fcst_horizon=learn.dls.d[-1], 
                                x_vars=['DST', 'F10'], y_vars=['DST', 'F10'])
X_combined.shape, y_combined.shape

((230497, 2, 144), (230497, 2, 144))

In [76]:
y_combined_test = y_combined[learn.dls[2].splits]
y_combined_test.shape

(45888, 2, 144)

In [81]:
#|export

# The threshold classify the forecasts of Dst are based on the values of the Dst itself
# and the solar activity level, as defined by the F10.7 (according to Licata et al.).
# Therefore, below is the table of all possible combined conditions to evaluate Dst.

# | Dst Level | Solar Level | Dst Value         | Solar Value       |
# |-----------|-------------|-------------------|-------------------|
# | G0        | Low         | Dst ≥ -30         | F 10.7 ≤ 75       |
# | G0        | Moderate    | Dst ≥ -30         | 75 < F 10.7 ≤ 150 |
# | G0        | Elevated    | Dst ≥ -30         | 150 < F 10.7 ≤ 190|
# | G0        | High        | Dst ≥ -30         | F 10.7 > 190      |
# | G1        | Low         | -30 > Dst ≥ -50   | F 10.7 ≤ 75       |
# | G1        | Moderate    | -30 > Dst ≥ -50   | 75 < F 10.7 ≤ 150 |
# | G1        | Elevated    | -30 > Dst ≥ -50   | 150 < F 10.7 ≤ 190|
# | G1        | High        | -30 > Dst ≥ -50   | F 10.7 > 190      |
# | G2        | Low         | -50 > Dst ≥ -90   | F 10.7 ≤ 75       |
# | G2        | Moderate    | -50 > Dst ≥ -90   | 75 < F 10.7 ≤ 150 |
# | G2        | Elevated    | -50 > Dst ≥ -90   | 150 < F 10.7 ≤ 190|
# | G2        | High        | -50 > Dst ≥ -90   | F 10.7 > 190      |
# | G3        | Low         | -90 > Dst ≥ -130  | F 10.7 ≤ 75       |
# | G3        | Moderate    | -90 > Dst ≥ -130  | 75 < F 10.7 ≤ 150 |
# | G3        | Elevated    | -90 > Dst ≥ -130  | 150 < F 10.7 ≤ 190|
# | G3        | High        | -90 > Dst ≥ -130  | F 10.7 > 190      |
# | G4        | Low         | -130 > Dst ≥ -350 | F 10.7 ≤ 75       |
# | G4        | Moderate    | -130 > Dst ≥ -350 | 75 < F 10.7 ≤ 150 |
# | G4        | Elevated    | -130 > Dst ≥ -350 | 150 < F 10.7 ≤ 190|
# | G4        | High        | -130 > Dst ≥ -350 | F 10.7 > 190      |
# | G5        | Low         | Dst ≤ -350        | F 10.7 ≤ 75       |
# | G5        | Moderate    | Dst ≤ -350        | 75 < F 10.7 ≤ 150 |
# | G5        | Elevated    | Dst ≤ -350        | 150 < F 10.7 ≤ 190|
# | G5        | High        | Dst ≤ -350        | F 10.7 > 190      |


# In order to create all those combined conditions, let's merge the data from Dst
# and F10.7. Since this data is provided with different frequencies (1 day for F10.7 
# and 1 hour for Dst, we will fill the missing timestamps of F10.7 with forward-filling)



def split_data_by_dst_f107(data):
    # function that splits the Dst data into all the possible Dst x F10.7 combinations
    # according to the thresholds defined above. # The decision is made based on 
    # the timestemp of each sample
    # The function returns a dictionary with the Dst x F10.7 combinations as keys
    # and the corresponding Dst data as values.
    # Input:
    # data: Dst data and F10 data (numpy array of shape (n_samples, 2, n_timesteps))
    # Output:
    # data_split: dictionary with the Dst x F10.7 combinations as keys, and the
    # corresponding Dst data as values.
    data_split = {}
    # Dst ≥ -30
    data_split['G0_Low'] = data[(data[:, 0, 0] >= -30) & (data[:, 1, 0] <= 75)]
    data_split['G0_Moderate'] = data[(data[:, 0, 0] >= -30) & (data[:, 1, 0] > 75) & (data[:, 1, 0] <= 150)]
    data_split['G0_Elevated'] = data[(data[:, 0, 0] >= -30) & (data[:, 1, 0] > 150) & (data[:, 1, 0] <= 190)]
    data_split['G0_High'] = data[(data[:, 0, 0] >= -30) & (data[:, 1, 0] > 190)]
    # -30 > Dst ≥ -50
    data_split['G1_Low'] = data[(data[:, 0, 0] < -30) & (data[:, 0, 0] >= -50) & (data[:, 1, 0] <= 75)]
    data_split['G1_Moderate'] = data[(data[:, 0, 0] < -30) & (data[:, 0, 0] >= -50) & (data[:, 1, 0] > 75) & (data[:, 1, 0] <= 150)]
    data_split['G1_Elevated'] = data[(data[:, 0, 0] < -30) & (data[:, 0, 0] >= -50) & (data[:, 1, 0] > 150) & (data[:, 1, 0] <= 190)]
    data_split['G1_High'] = data[(data[:, 0, 0] < -30) & (data[:, 0, 0] >= -50) & (data[:, 1, 0] > 190)]
    # -50 > Dst ≥ -90
    data_split['G2_Low'] = data[(data[:, 0, 0] < -50) & (data[:, 0, 0] >= -90) & (data[:, 1, 0] <= 75)]
    data_split['G2_Moderate'] = data[(data[:, 0, 0] < -50) & (data[:, 0, 0] >= -90) & (data[:, 1, 0] > 75) & (data[:, 1, 0] <= 150)]
    data_split['G2_Elevated'] = data[(data[:, 0, 0] < -50) & (data[:, 0, 0] >= -90) & (data[:, 1, 0] > 150) & (data[:, 1, 0] <= 190)]
    data_split['G2_High'] = data[(data[:, 0, 0] < -50) & (data[:, 0, 0] >= -90) & (data[:, 1, 0] > 190)]
    # -90 > Dst ≥ -130
    data_split['G3_Low'] = data[(data[:, 0, 0] < -90) & (data[:, 0, 0] >= -130) & (data[:, 1, 0] <= 75)]
    data_split['G3_Moderate'] = data[(data[:, 0, 0] < -90) & (data[:, 0, 0] >= -130) & (data[:, 1, 0] > 75) & (data[:, 1, 0] <= 150)]
    data_split['G3_Elevated'] = data[(data[:, 0, 0] < -90) & (data[:, 0, 0] >= -130) & (data[:, 1, 0] > 150) & (data[:, 1, 0] <= 190)]
    data_split['G3_High'] = data[(data[:, 0, 0] < -90) & (data[:, 0, 0] >= -130) & (data[:, 1, 0] > 190)]
    # -130 > Dst ≥ -350
    data_split['G4_Low'] = data[(data[:, 0, 0] < -130) & (data[:, 0, 0] >= -350) & (data[:, 1, 0] <= 75)]
    data_split['G4_Moderate'] = data[(data[:, 0, 0] < -130) & (data[:, 0, 0] >= -350) & (data[:, 1, 0] > 75) & (data[:, 1, 0] <= 150)]
    data_split['G4_Elevated'] = data[(data[:, 0, 0] < -130) & (data[:, 0, 0] >= -350) & (data[:, 1, 0] > 150) & (data[:, 1, 0] <= 190)]
    data_split['G4_High'] = data[(data[:, 0, 0] < -130) & (data[:, 0, 0] >= -350) & (data[:, 1, 0] > 190)]
    # Dst < -350
    data_split['G5_Low'] = data[(data[:, 0, 0] < -350) & (data[:, 1, 0] <= 75)]
    data_split['G5_Moderate'] = data[(data[:, 0, 0] < -350) & (data[:, 1, 0] > 75) & (data[:, 1, 0] <= 150)]
    data_split['G5_Elevated'] = data[(data[:, 0, 0] < -350) & (data[:, 1, 0] > 150) & (data[:, 1, 0] <= 190)]
    data_split['G5_High'] = data[(data[:, 0, 0] < -350) & (data[:, 1, 0] > 190)]
    return data_split
    

In [86]:
# Test
y_combined_test_split = split_data_by_dst_f107(y_combined_test)

# Check the shape of each key
for key in y_combined_test_split.keys():
    print(key, y_combined_test_split[key].shape)

G0_Low (5667, 2, 144)
G0_Moderate (29957, 2, 144)
G0_Elevated (4248, 2, 144)
G0_High (478, 2, 144)
G1_Low (190, 2, 144)
G1_Moderate (3240, 2, 144)
G1_Elevated (544, 2, 144)
G1_High (74, 2, 144)
G2_Low (24, 2, 144)
G2_Moderate (1064, 2, 144)
G2_Elevated (164, 2, 144)
G2_High (0, 2, 144)
G3_Low (0, 2, 144)
G3_Moderate (190, 2, 144)
G3_Elevated (12, 2, 144)
G3_High (0, 2, 144)
G4_Low (0, 2, 144)
G4_Moderate (36, 2, 144)
G4_Elevated (0, 2, 144)
G4_High (0, 2, 144)
G5_Low (0, 2, 144)
G5_Moderate (0, 2, 144)
G5_Elevated (0, 2, 144)
G5_High (0, 2, 144)


In [87]:
# Take only the first variable of each key (the Dst)
y_test_split = {}
for key in y_combined_test_split.keys():
    y_test_split[key] = y_combined_test_split[key][:, 0, :]